In [9]:
import os
a4_path = "csai_a4"
# print cwd
print(os.getcwd())
# move cwd to csai_a4
os.chdir(a4_path)
stim_path = "CSI01_stim_lists.txt"
# opena nd load the file
with open(stim_path, 'r') as f:
    stim_list = f.read().splitlines()
file_set = []
indices_list = []
for i in range(len(stim_list)):
    file = stim_list[i]
    # first 3 chars of the file name is not rep, add to the set
    if file[:3] != "rep":
        indices_list.append(i)
        file_set.append(file)
print(len(indices_list))
fin_images = []
for i in indices_list:
    fin_images.append(stim_list[i])
for file in fin_images:
    # if file has first 3 chars as rep, print file name
    if file[:3] == "rep":
        print(file)
print(len(file_set))
print(stim_list[0:20])
# for coco files: first 4 chars are COCO
# for imagenet files: first 2 chars are n0 or n1
# rest are Scene files
coco_files = []
imagenet_files = []
scene_files = []
for i in range(len(stim_list)):
    if stim_list[i][0:4] == "COCO":
        #print(stim_list[i][0:4], stim_list[i], "coco")
        coco_files.append(stim_list[i])
    elif stim_list[i][0:2] == "n0" or stim_list[i][0:2] == "n1":
        #print(stim_list[i][0:2], stim_list[i], "imagenet")
        imagenet_files.append(stim_list[i])
    else:
        #print(stim_list[i], "scene")
        scene_files.append(stim_list[i] )
print(coco_files[0], scene_files[0], imagenet_files[0])
coco_dir = "presented_stimuli/COCO/"
imagenet_dir = "presented_stimuli/ImageNet/"
scene_dir = "presented_stimuli/Scene/"
# check if these directories exist
if not os.path.exists(coco_dir):
    print("COCO directory does not exist")
if not os.path.exists(imagenet_dir):
    print("Imagenet directory does not exist")
if not os.path.exists(scene_dir):
    print("Scene directory does not exist")
root_path = "CSI1/h5/"
# load all the files in the root path into list
list_files = []
for root, dirs, files in os.walk(root_path):
    for file in files:
        list_files.append(file)

print(list_files)
# load file CSI1/h5/CSI1_ROIs_TR1.h5
!pip install h5py
import h5py
path = "CSI1/h5/CSI1_ROIs_TR34.h5"
f = h5py.File(path, 'r')





/home2/advaith.malladi/csai_a4


FileNotFoundError: [Errno 2] No such file or directory: 'csai_a4'

### Loading FMRI Data and Removing Repetitions

In [2]:
import numpy as np
keys = list(f.keys())
for key in keys:
    print(key, f[key].shape)
# concat LHLOC and RHLOC along dim 1
lhloc = f["LHLOC"]
rhloc = f["RHLOC"]
lhloc.shape, rhloc.shape
loc = np.concatenate([lhloc, rhloc], axis=1)
loc.shape
# same for OPA 
lhopa = f["LHOPA"]
rhopa = f["RHOPA"]
opa = np.concatenate([lhopa, rhopa], axis=1)
# same for PPA 
lhppa = f["LHPPA"]
rhppa = f["RHPPA"]
ppa = np.concatenate([lhppa, rhppa], axis=1)
# same for RSC
lhrsc = f["LHRSC"]
rhrsc = f["RHRSC"]
rsc = np.concatenate([lhrsc, rhrsc], axis=1)
# same for EarlyVis
lhearlyvis = f["LHEarlyVis"]
rhearlyvis = f["RHEarlyVis"]
earlyvis = np.concatenate([lhearlyvis, rhearlyvis], axis=1)
print(loc.shape, opa.shape, ppa.shape, rsc.shape, earlyvis.shape)
print(len(indices_list), len(fin_images))
# in array loc, include only the indices in indices_list
loc_final = []
for i in indices_list:
    loc_final.append(loc[i])
loc_final = np.array(loc_final)
# do the same for opa, ppa, rsc, earlyvis
opa_final = []
for i in indices_list:
    opa_final.append(opa[i])
opa_final = np.array(opa_final)
ppa_final = []
for i in indices_list:
    ppa_final.append(ppa[i])
ppa_final = np.array(ppa_final)
rsc_final = []
for i in indices_list:
    rsc_final.append(rsc[i])
rsc_final = np.array(rsc_final)
earlyvis_final = []
for i in indices_list:
    earlyvis_final.append(earlyvis[i])
earlyvis_final = np.array(earlyvis_final)
print(loc_final.shape, opa_final.shape, ppa_final.shape, rsc_final.shape, earlyvis_final.shape)


LHEarlyVis (5254, 210)
LHLOC (5254, 152)
LHOPA (5254, 101)
LHPPA (5254, 131)
LHRSC (5254, 86)
RHEarlyVis (5254, 285)
RHLOC (5254, 190)
RHOPA (5254, 187)
RHPPA (5254, 200)
RHRSC (5254, 143)
(5254, 342) (5254, 288) (5254, 331) (5254, 229) (5254, 495)
4916 4916
(4916, 342) (4916, 288) (4916, 331) (4916, 229) (4916, 495)


### Stimulus Loading and Loading Model

In [3]:
import glob
from pathlib import Path
from PIL import Image
from tqdm import tqdm, trange
import pickle
import wget
import zipfile

import numpy as np
from scipy import stats
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
filename = fin_images[0]

/home2/advaith.malladi/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from transformers import ViTImageProcessor, ViTModel, AutoImageProcessor
from PIL import Image
from torch import nn, optim
class ptuned_VIT(nn.Module):
    def __init__(self, num_classes):
        super(ptuned_VIT, self).__init__()
        self.model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
        self.embeddings = nn.Embedding(3, 768)
        self.classification_layer = nn.Linear(768, num_classes)
        self.softmax = nn.Softmax(dim=1)
        self.trainable = [self.embeddings, self.classification_layer]
        for param in self.model.parameters():
            param.requires_grad = False
    def forward(self, x):
        tens = [0,1,2]
        tens = torch.tensor(tens)
        x1 = self.embeddings(tens)
        x2 = self.model.embeddings(x)
        # change x1 shape from x, 768 to 1, x, 768
        x1 = x1.unsqueeze(0)
        # concat x1 and x2 along the first dimension
        # x1 is 1, 3, 768, make is x2.shape[0], 3, 768
        x1 = x1.expand(x2.shape[0], -1, -1)
        x = torch.cat((x1, x2), 1)
        x = self.model.encoder(x)
        x = x['last_hidden_state']
        x1 = x
        x = self.model.pooler(x)
        x = self.classification_layer(x)
        x = self.softmax(x)
        return x, x1
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
mnist_pt = torch.load("pt_vit_mnist_model.pt")
cifar_pt = torch.load("pt_vit_cifar_model.pt")

In [5]:
test_file = open("test.txt", "w")
from tqdm import tqdm
import regex as re

def get_feat(files):
    mnist_feats = []
    cifar_feats = []
    coco_dir = "presented_stimuli/COCO/"
    imagenet_dir = "presented_stimuli/ImageNet/"
    scene_dir = "presented_stimuli/Scene/"
    new_files = []
    for file in files:
        if file[0:4] == "COCO":
            new_path = coco_dir + file
        elif file[0:2] == "n0" or file[0:2] == "n1":
            new_path = imagenet_dir + file
        else:
            new_path = scene_dir + file
        new_files.append(new_path)
    mnist_pt.to("cpu")
    for filename in tqdm(new_files, total = len(new_files)):
        img = Image.open(filename)
        img = processor(img, return_tensors="pt")
        img = img['pixel_values']
        img = img.to("cpu")
        logits, feats = mnist_pt(img)
        # convert from 1, a, b to a, b
        feats = feats.squeeze(0)
        feats_mnist = torch.mean(feats, dim=0)
        feats_mnist = feats_mnist.detach().numpy()
        mnist_feats.append(feats_mnist)
        cifar_pt.to("cpu")
        logits, feats = cifar_pt(img)
        feats = feats.squeeze(0)
        feats_cifar = torch.mean(feats, dim=0)
        feats_cifar = feats_cifar.detach().numpy()
        cifar_feats.append(feats_cifar)
    mnist_feats = np.array(mnist_feats)
    cifar_feats = np.array(cifar_feats)
    return mnist_feats, cifar_feats

    
mnist_feats, cifar_feats = get_feat(fin_images)
            

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4916/4916 [08:53<00:00,  9.21it/s]


### Brain Decoding

In [6]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
# import R2 as well
from sklearn.metrics import r2_score
def cos_d(y1, y2):
    cos_sim = np.dot(y1, y2)/(np.linalg.norm(y1)*np.linalg.norm(y2))
    cos_d = 1 - cos_sim
    return cos_d
def get_2v2_accuracy(y_pred, y_test):
    N = y_pred.shape[0]
    tot_cnt = 0
    pos_cnt = 0
    for i in range(0, N):
        for j in range(1, N):
            yi_test = y_test[i]
            yi_pred = y_pred[i]
            yj_test = y_test[j]
            yj_pred = y_pred[j]
            score1 = cos_d(yi_test, yi_pred) + cos_d(yj_test, yj_pred)
            score2 = cos_d(yi_test, yj_pred) + cos_d(yj_test, yi_pred)
            if score1 < score2:
                pos_cnt += 1
            tot_cnt += 1
    return pos_cnt/tot_cnt

def get_avg_corr(y_pred, y_test):
    N = y_pred.shape[0]
    tot_corr = 0
    for i in range(0, N):
        corr = np.corrcoef(y_pred[i], y_test[i])[0,1]
        tot_corr += corr
    return tot_corr/N

def decoding_module(X, y):
    kf = KFold(n_splits=5)
    mse = []
    r2 = []
    acc = []
    acc_scores = []
    corr_scores = []
    for train_index, test_index in  kf.split(X) :
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model = Ridge()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        get_2v2_accuracy(y_pred, y_test)
        mse.append(mean_squared_error(y_test, y_pred))
        r2.append(r2_score(y_test, y_pred))
        acc_scores.append(get_2v2_accuracy(y_pred, y_test))
        corr_scores.append(get_avg_corr(y_pred, y_test))
    loc_dict = {}
    loc_dict["MSE"] = np.mean(mse)
    loc_dict["R2"] = np.mean(r2)
    loc_dict["Accuracy"] = np.mean(acc_scores)
    loc_dict["Correlation"] = np.mean(corr_scores)
    return loc_dict



In [7]:
feats = [mnist_feats, cifar_feats]
feat_names = ["mnist", "cifar"]
rois = [loc_final, opa_final, ppa_final, rsc_final, earlyvis_final]
roi_names = ["LOC", "OPA", "PPA", "RSC", "EarlyVis"]
decoding_results = {}
for i in range(len(feats)):
    for j in range(len(rois)):
        str1 = f"Decoding {feat_names[i]} using {roi_names[j]}"
        loc_feat = feats[i]
        loc_roi = rois[j]
        x = loc_roi
        y = loc_feat
        loc_results = decoding_module(x,y)
        print(loc_results)
        decoding_results[str1] = loc_results

{'MSE': 46.3713631634123, 'R2': 0.002492346152606733, 'Accuracy': 0.7562787163037682, 'Correlation': 0.6224184502389152}
{'MSE': 46.10152688761668, 'R2': 0.004675018122699026, 'Accuracy': 0.7495915916565418, 'Correlation': 0.6241753485593234}
{'MSE': 46.64586474428514, 'R2': -0.00671368236643888, 'Accuracy': 0.7187971272060846, 'Correlation': 0.6183939282539741}
{'MSE': 46.49262379839511, 'R2': -0.002334279737623587, 'Accuracy': 0.6835862199911704, 'Correlation': 0.6206345995111626}
{'MSE': 46.473650719804375, 'R2': -0.002213545636266707, 'Accuracy': 0.7030619466233033, 'Correlation': 0.6203937039943589}
{'MSE': 46.70773813906378, 'R2': 0.0023301087906300194, 'Accuracy': 0.7544310246858997, 'Correlation': 0.6225831668853784}
{'MSE': 46.42708902266624, 'R2': 0.0045523580480065304, 'Accuracy': 0.7477646462297923, 'Correlation': 0.6243655380481512}
{'MSE': 46.975741357705004, 'R2': -0.0068418197380436, 'Accuracy': 0.7166065521933406, 'Correlation': 0.6185793279790157}
{'MSE': 46.818912669

In [8]:
feats = [mnist_feats, cifar_feats]
feat_names = ["mnist", "cifar"]
rois = [loc_final, opa_final, ppa_final, rsc_final, earlyvis_final]
roi_names = ["LOC", "OPA", "PPA", "RSC", "EarlyVis"]
encoding_results = {}
for i in range(len(feats)):
    for j in range(len(rois)):
        str1 = f"Decoding {feat_names[i]} using {roi_names[j]}"
        loc_feat = feats[i]
        loc_roi = rois[j]
        y = loc_roi
        x = loc_feat
        loc_results = decoding_module(x,y)
        print(loc_results)
        encoding_results[str1] = loc_results

{'MSE': 0.00012173177579614387, 'R2': -0.20656616415260465, 'Accuracy': 0.7058057216752819, 'Correlation': 0.19876371096776418}
{'MSE': 9.074416686662823e-05, 'R2': -0.16161847518649294, 'Accuracy': 0.7594152030884718, 'Correlation': 0.26390564970825015}
{'MSE': 0.0003084225118174564, 'R2': -0.18651588344267048, 'Accuracy': 0.7384781920704903, 'Correlation': 0.1461920116518157}
{'MSE': 0.0002546602404998866, 'R2': -0.16634132673139762, 'Accuracy': 0.6822310500672949, 'Correlation': 0.09629421117604631}
{'MSE': 0.00015525178506046043, 'R2': -0.22954436483387325, 'Accuracy': 0.6396903132902682, 'Correlation': 0.20718234999451032}
{'MSE': 0.0001217564987661722, 'R2': -0.20690190543596937, 'Accuracy': 0.7062302789061252, 'Correlation': 0.19895001853764083}
{'MSE': 9.078429845725822e-05, 'R2': -0.16210771280006023, 'Accuracy': 0.75909401465501, 'Correlation': 0.26383820224223786}
{'MSE': 0.00030851803168683304, 'R2': -0.1866873353299835, 'Accuracy': 0.7376128364974085, 'Correlation': 0.1459

In [12]:
# save encoding_results as encoding_ft_vis.json and indent 4
import json
with open("encoding_pt_vis.json", "w") as f:
    json.dump(encoding_results, f, indent=4)

In [13]:
# same for decoding
with open("decoding_pt_vis.json", "w") as f:
    json.dump(decoding_results, f, indent=4)

In [14]:
# print cwd
print(os.getcwd())

/home2/advaith.malladi/csai_a4
